In [5]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET

In [ ]:
# Function to load images and annotations from our dataset
def load_data(split):
    images = []
    annotations = []

    images_dir = os.path.join(dataset_path, split, "images")
    annotations_dir = os.path.join(dataset_path, split, "annotations")

    for filename in os.listdir(images_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(images_dir, filename)
            image = cv2.imread(image_path)
            images.append(image)

            annotation_path = os.path.join(annotations_dir, filename.split(".")[0] + ".xml")
            tree = ET.parse(annotation_path)
            root = tree.getroot()

            annotation = []
            for obj in root.findall("object"):
                name = obj.find("name").text
                bbox = obj.find("bndbox")
                xmin = int(bbox.find("xmin").text)
                ymin = int(bbox.find("ymin").text)
                xmax = int(bbox.find("xmax").text)
                ymax = int(bbox.find("ymax").text)
                annotation.append((name, xmin, ymin, xmax, ymax))

            annotations.append(annotation)

    return images, annotations